In [ ]:
%pip install --upgrade --force-reinstall -r https://aka.ms/automl_linux_requirements.txt

In [ ]:
#pip install --upgrade azureml-automl-core

In [ ]:
import azureml.core

print("SDK Version:", azureml.core.VERSION)


In [ ]:
# define widgets - NEED TO DEFINE IT ONCE
# dynamic variables (pass it from ADF)
# first time runtime parameter
# dbutils.widgets.dropdown("environment", "dev", ["dev","uat","prod"])
# dbutils.widgets.dropdown("new_training", "False", ["True","False"])
# dbutils.widgets.dropdown("system_name", "bimodelapi", ["bimodelapi"])
# dbutils.widgets.text("system_name", "","")
# dbutils.widgets.remove("pbiapi")
environment = dbutils.widgets.get("environment")
new_training = dbutils.widgets.get("new_training")
system_name = dbutils.widgets.get("system_name")
if environment in {"prod"}:
    aml_compute_cluster_name = "cc-bi-ml-prod01"
else:
    aml_compute_cluster_name = "cc-bi-ml-devqa01"

print(environment, system_name, new_training, aml_compute_cluster_name)

In [ ]:
%run ./data_prep_rev_region_forecast

In [ ]:

# Import the Workspace class and configure your local envionment
from azureml.core import Workspace

# from azureml.core.authentication import InteractiveLoginAuthentication
import os
from azureml.core.authentication import ServicePrincipalAuthentication

tenant_id = "cfa930ff-a3f0-4933-8cea-744f8ead7682"
service_principal_id = "626c2c00-26de-46ee-a20d-6027445518ff"

svc_pr_password = dbutils.secrets.get(
    scope="kv-bi-devqa-01-secrets", key="dbw-azureml-devqa-key"
)

svc_pr = ServicePrincipalAuthentication(
    tenant_id=tenant_id,
    service_principal_id=service_principal_id,
    service_principal_password=svc_pr_password,
)

# Create Workspace if required
"""
from azureml.core import Workspace

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,                      
                      exist_ok=True)
ws.get_details()

"""

try:
    ws = Workspace(
        workspace_name=workspace_name,
        subscription_id=subscription_id,
        resource_group=resource_group,
        auth=svc_pr,  # forced_interactive_auth
    )

    # Persist the subscription id, resource group name, and workspace name in aml_config/config.json.
    ws.write_config()
    print(ws)
    print("Found workspace {} at location {}".format(ws.name, ws.location))
    # ws.write_config(path="./aml_config/",file_name="ws_config.json")
    # use the get method to load an existing workspace without using configuration files.
except Exception as error:
    print(error)
    log_error("{} {}".format(notebook, error)) #log error in sentry
    raise dbutils.notebook.exit(error) #raise the exception



In [ ]:

# Create an experiement
import os
import random
import time
import json

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow

import azureml.core
import pandas as pd
import numpy as np
import logging

from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
from azureml.automl.core.featurization import FeaturizationConfig

# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/automated-machine-learning/forecasting-orange-juice-sales/auto-ml-forecasting-orange-juice-sales.ipynb
# Choose a name for the experiment and specify the project folder.
experiment_name = "automl-revenue-region-forecast"

try:

    experiment = Experiment(ws, experiment_name)

    output = {}
    output["SDK version"] = azureml.core.VERSION
    output["Subscription ID"] = ws.subscription_id
    output["Workspace Name"] = ws.name
    output["Resource Group"] = ws.resource_group
    output["Location"] = ws.location
    output["Experiment Name"] = experiment.name
    pd.set_option("display.max_colwidth", -1)
    print(pd.DataFrame(data=output, index=[""]).T)
except Exception as error:
    print(error)
    log_error("{} {}".format(notebook, error)) #log error in sentry
    raise dbutils.notebook.exit(error) #raise the exception
    
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU clustera
try:
    amlcompute_cluster_name = aml_compute_cluster_name

    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print("Found existing cluster, use it:", amlcompute_cluster_name)
except Exception as error:
    print(error)
    log_error("{} {}".format(notebook, error)) #log error in sentry
    raise dbutils.notebook.exit(error) #raise the exception



In [ ]:

# Data Split
# split the data into a training and a testing set for later forecast evaluation. The test set will contain the final test_size months of observed sales for each time-series.
# The splits should be stratified by series, so we use a group-by statement on the time series identifier columns.
# test_size = .11 #in percentage
# n_test_periods =  int(tseries * test_size)
# n_test_periods = relative_offset_future_period

df = merge_final.copy()
'''
tseries = df.groupby(time_column_name).ngroups
print("Data contains {0} individual time_column_name.".format(tseries))
test_size = round(n_test_periods / tseries, 2)
print("Time Column Name:", time_column_name)
print("Test Size: ", test_size, "Test Periods: ", n_test_periods)
print(df[time_column_name].unique())

#print("n_test_periods: ", n_test_periods)
'''

train = df.query("Relative_Month_Offset < 0")
test = df.drop(train.index)

#(train, test) = split_last_n_by_series_id(df, n_test_periods)

print(df.shape)
print(train.shape)
print(test.shape)

train.to_csv(output_data_path + "revregionforecast_train.csv", index=None, header=True)
test.to_csv(output_data_path + "revregionforecast_test.csv", index=None, header=True)

In [ ]:

# Summary Stats for Train and Test dataframes

print("Train************************")
#print(train.info(verbose=True))
train.describe(include="all").transpose().head()
#train.tail(10)



In [ ]:

print("Test************************")
#print(test.info(verbose=True))
test.describe(include="all").transpose().head()
#test.tail(10)



In [ ]:

# plot the example time series
#!pip install seaborn
"""
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.float_format', lambda x: '%.4f' % x)

# sns.set_context('paper', font_scale=1.3)
# sns.set_style('white')

(fig, ax) = plt.subplots(figsize=(24, 9))
import matplotlib.pyplot as plt
whole_data = train.copy()
target_label = target_column_name
whole_data[target_label] = train[target_column_name]
for g in whole_data.groupby(time_series_id_column_names):
    plt.plot(g[1][time_column_name].values,
             g[1][target_column_name].values, label=g[0])
plt.legend()
#plt.show()

# plot the example time series

(fig, ax) = plt.subplots(figsize=(24, 9))
import matplotlib.pyplot as plt
whole_data = test.copy()
target_label = target_column_name
whole_data[target_label] = test[target_column_name]
for g in whole_data.groupby(time_series_id_column_names):
    plt.plot(g[1][time_column_name].values,
             g[1][target_column_name].values, label=g[0])
plt.legend()
#plt.show()
"""



In [ ]:

# Create dataset for training

from azureml.core import Dataset, Datastore

datastore = ws.get_default_datastore()
print("Default datastore's name: {}".format(datastore.name))

# register_spark_dataframe(dataframe=train_sdf, target=blobstore_datadir, name="revforecast_train.parquet", description=None, tags=None, show_progress=True)

datastore.upload_files(
    files=[
        output_data_path + "revregionforecast_train.csv",
        output_data_path + "revregionforecast_test.csv",
    ],
    target_path=blobstore_datadir,
    overwrite=True,
    show_progress=True,
)

dataset_filename = blobstore_datadir + "revregionforecast_train.csv"
train_dataset = Dataset.Tabular.from_delimited_files(
    path=[(datastore, dataset_filename)]
)

#display(train_dataset.to_pandas_dataframe().head())
display(train_dataset.to_pandas_dataframe().tail())



In [ ]:
# Set forecasting_parameters for training

from azureml.automl.core.forecasting_parameters import ForecastingParameters

target_lag = "auto"  # [x for x in range(1,7)] #'auto'
window_size = "auto"
feature_lag = "auto"
forecast_horizon = n_test_periods

print("past_period", past_period)
print("n_test_periods", n_test_periods)
print("target_lags:", target_lag)
print("target_rolling_window_size:", window_size)
print("forecast_horizon:", forecast_horizon)
print("feature_lags:", feature_lag)

forecasting_parameters = ForecastingParameters(
    time_column_name=time_column_name,
    forecast_horizon=forecast_horizon,
    time_series_id_column_names=time_series_id_column_names,
    target_lags=target_lag,
    feature_lags=feature_lag,
    target_rolling_window_size=window_size# ,    drop_column_names = ['Billable_Headcount']
)

# drop_column_names = ['-1M Pipeline']

automl_config = AutoMLConfig(  # featurization_config,
    task="forecasting",
    debug_log="rev_region_forecast_errors.log",
    primary_metric="normalized_root_mean_squared_error",
    experiment_timeout_hours=4,
    training_data=train_dataset,
    label_column_name=target_column_name,
    enable_early_stopping=True,
    #spark_context=sc, #enable this for databricks cluster
    compute_target=compute_target,  # enable this for ml cluster
    enable_dnn=True,  # enable this for ml cluster
    featurization="auto",
    n_cross_validations=5,
    verbosity=logging.INFO,
    max_concurrent_iterations=16,
    max_cores_per_iteration=-1,
    forecasting_parameters=forecasting_parameters,
)

In [ ]:

# submit a new training run

try:
    if new_training == "True":
        print("New Training Run")
        remote_run = experiment.submit(automl_config, show_output=True)
    else:
        # If you need to retrieve a run that already started, use the following code
        print("Existing Training Run")
        from azureml.train.automl.run import AutoMLRun

        remote_run = AutoMLRun(
            experiment=experiment, run_id = "AutoML_04c91504-6d90-4eed-9595-32b363541746"
        )
        
except Exception as error:
    print(error)
    log_error("{} {}".format(notebook, error)) #log error in sentry
    raise dbutils.notebook.exit(error) #raise the exception

remote_run

In [ ]:

# Get run_id and run_datetime
rr = remote_run.wait_for_completion()
run_id = rr.get("runId")
run_datetime = rr.get("endTimeUtc")
print(run_id, run_datetime)

# Retrieve the Best Model
try:
    best_run, fitted_model = remote_run.get_output()
    print("Best Run Model: ", best_run)
    print(fitted_model.steps)
    model_name = best_run.properties["model_name"]
except Exception as error:
    print(error)
    log_error("{} {}".format(notebook, error)) #log error in sentry
    raise dbutils.notebook.exit(error) #raise the exception
# print ('Model Name: ', model_name)



In [ ]:

# Transparency
# View updated featurization summary
try:
    featurization_summary = fitted_model.named_steps[
        "timeseriestransformer"
    ].get_featurization_summary()

    # View the featurization summary as a pandas dataframe

    fs = pd.DataFrame.from_records(featurization_summary)

except Exception as error:
    print(error)
    log_error("{} {}".format(notebook, error)) #log error in sentry
    raise dbutils.notebook.exit(error) #raise the exception

fs



In [ ]:

# Get metrics for best run
pd.set_option("display.float_format", lambda x: "%.5f" % x)
metricslist = {}
properties = best_run.get_properties()
# print(properties)
metrics = {k: v for k, v in best_run.get_metrics().items() if isinstance(v, float)}
metricslist[int(properties["iteration"])] = metrics
rundata = pd.DataFrame.from_records(metricslist).sort_index(1)

rundata



In [ ]:

# FORECASTING

X_test = test.copy()
y_test = X_test.pop(target_column_name).values

# forecast returns the predictions and the featurized data, aligned to X_test.
# This contains the assumptions that were made in the forecast
# The featurized data, aligned to y, will also be returned.
# and helps align the forecast to the original data
try:
    y_predictions, X_trans = fitted_model.forecast(X_test)

    # from forecasting_helper import align_outputs

    df_all = align_outputs(y_predictions, X_trans, X_test, y_test, target_column_name)
    df_all.rename(columns={"predicted": "Predicted_Revenue"}, inplace=True)

except Exception as error:
    print(error)
    log_error("{} {}".format(notebook, error)) #log error in sentry
    raise dbutils.notebook.exit(error) #raise the exception

df_all.info()

In [ ]:

# Evaluate
# To evaluate the accuracy of the forecast, we'll compare against the actual sales quantities for some select metrics, included the mean absolute percentage error (MAPE).

# We'll add predictions and actuals into a single dataframe for convenience in calculating the metrics.

assign_dict = {'Predicted_Revenue': y_predictions, target_column_name: y_test}
df_all = X_test.assign(**assign_dict)
df_all.tail(10)

from azureml.automl.core.shared import constants
from azureml.automl.runtime.shared.score import scoring
from matplotlib import pyplot as plt

# use automl scoring module

scores = scoring.score_regression(
    y_test=df_all[target_column_name],
    y_pred=df_all["Predicted_Revenue"],
    metrics=list(constants.Metric.SCALAR_REGRESSION_SET),
)

print("[Test data scores]")
for key, value in scores.items():
    print("{}:   {:.3f}".format(key, value))

# Plot outputs

#%matplotlib inline
test_pred = plt.scatter(
    df_all[target_column_name], df_all["Predicted_Revenue"], color="b"
)
test_test = plt.scatter(
    df_all[target_column_name], df_all[target_column_name], color="g"
)
plt.legend(
    (test_pred, test_test), ("prediction", "truth"), loc="upper left", fontsize=8
)
plt.show()



In [ ]:

# Print metrics by different groups

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


if "horizon_origin" in df_all.columns:
    group_by = "horizon_origin"
else:
    group_by = "Relative_Offset"

df_mpae = df_all.groupby(group_by).apply(
    lambda df: pd.Series(
        {
            "MAPE": MAPE(df[target_column_name], df["Predicted_Revenue"]),
            "RMSE": np.sqrt(
                mean_squared_error(df[target_column_name], df["Predicted_Revenue"])
            ),
            "MAE": mean_absolute_error(df[target_column_name], df["Predicted_Revenue"]),
            "R2_SCORE": r2_score(df[target_column_name], df["Predicted_Revenue"]),
        }
    )
)
display(df_mpae)

# Box plot with different groups
df_all_MAPE = df_all.assign(
    MAPE = MAPE(df_all[target_column_name], df_all["Predicted_Revenue"])
)

MAPEs = [df_all_MAPE[df_all[group_by] == h].MAPE.values for h in range(1, forecast_horizon + 1)]


#%matplotlib inline
plt.boxplot(MAPEs)
plt.yscale("log")
plt.xlabel(group_by)
plt.ylabel("MAPE (%)")
plt.title("Mean Absolute Percentage Errors")

plt.show()


In [ ]:

# Confidence intervals
"""
quantiles = fitted_model.forecast_quantiles(X_test)
# quantiles.tail(10)

# specify which quantiles you would like

fitted_model.quantiles = [0.01, 0.5, 0.8, 0.95]

# use forecast_quantiles function, not the forecast() one

y_pred_quantiles =  fitted_model.forecast_quantiles(X_test)

# quantile forecasts returned in a Dataframe along with the time and time series id columns 

#y_pred_quantiles.tail(10)
"""



In [ ]:

# Merge df_all with train

from functools import reduce

# Get the column index list in the right order

cols_sort_list = list(df_all.columns)
#print(cols_sort_list)
# df_all.tail(10)

nan_value = 0
merge_dfs = [train, df_all]

final_merge_df = train.append(df_all, ignore_index=True, sort=False).sort_values(
    by=sort_cols_snp
)

final_merge_df = final_merge_df.replace(np.nan, 0, regex=True)

final_merge_df["Relative_Month_Offset"] = round(
    (final_merge_df["End_of_Month"] - current_eom) / np.timedelta64(1, "M"), 0
).astype(int)

if "origin" in final_merge_df.columns:
    final_merge_df["origin"] = pd.to_datetime(final_merge_df["origin"])

    # .dt.date

# sort using original cols_sort_list

final_merge_df = final_merge_df.sort_values(by=sort_cols_snp).reset_index(drop=True)
final_merge_df = final_merge_df.reindex(columns=cols_sort_list)

# add new cols

final_merge_df.insert(0,"Forecast_Date", pd.to_datetime(run_datetime).normalize().date())
final_merge_df["Forecast_Date"] = pd.to_datetime(
    final_merge_df["Forecast_Date"]
)  # .dt.date
final_merge_df["Run_ID"] = run_id

move_column_inplace(final_merge_df, 'Predicted_Revenue', 9)
move_column_inplace(final_merge_df, 'Revenue', 10)

'''
final_merge_df["Predicted_Revenue_Variance"] = np.where(
    final_merge_df["Relative_Month_Offset"] < 0,
    final_merge_df["Revenue"] - final_merge_df["Predicted_Revenue"],
    np.nan,
)
final_merge_df["Predicted_Revenue_Variance_Percent"] = np.where(
    final_merge_df["Relative_Month_Offset"] < 0,
    round(
        final_merge_df["Predicted_Revenue_Variance"]
        / final_merge_df["Predicted_Revenue"]
        * 100,
        ndigits=4,
    ),
    np.nan,
)
final_merge_df["Predicted_Revenue_Variance"].where(
    final_merge_df["Predicted_Revenue"] != 0, np.nan, inplace=True
)
final_merge_df["Predicted_Revenue_Variance_Percent"].where(
    final_merge_df["Predicted_Revenue"] != 0, np.nan, inplace=True
)
final_merge_df["Predicted_Revenue_Variance_Percent"] = round(
    final_merge_df["Predicted_Revenue_Variance_Percent"], ndigits=4
)
'''

new_cols = ["Predicted_Revenue"]

numeric_cols_final_merge = numeric_cols + new_cols
#if "Billable_Headcount" in numeric_cols_final_merge:
#    numeric_cols_final_merge.remove("Billable_Headcount")
#print(numeric_cols_final_merge)

numeric_cols = numeric_cols_final_merge
final_merge_df = convert_date_cols(final_merge_df)
final_merge_df = coerce_to_numeric(final_merge_df)
final_merge_df = coerce_to_int(final_merge_df)

print(" final_merge_df: ")
display(final_merge_df.tail())
final_merge_df.to_csv(output_data_path + "final_merge_df.csv", index=False)
final_merge_df.to_parquet(output_data_path + "final_merge_df.parquet", index=None)


In [ ]:

# pivot by EOM1, Fin_Entity_ID
pd.set_option("display.float_format", lambda x: "%.2f" % x)  #

#final_cond = "Relative_Snapshot_Month_Offset == 0 and Snp_Seq_No == 6"

final_merge_df1 = final_merge_df.copy() #query(final_cond)

cols = [
    "Predicted_Revenue",
    "Revenue"]
# final_cond = "Relative_Offset >= 0 and Relative_Snapshot_Month_Offset <= -3"
# final_merge_df1 = final_merge_df.query(final_cond)

final_merge_df1 = convert_date_cols(final_merge_df1)

final_merge_pivot1 = final_merge_df1.pivot_table(
    index=[
        "Forecast_Date",
        "Snapshot_Date",
        "End_of_Month",
        "Relative_Snapshot_Month_Offset",
        "Relative_Month_Offset",
        "Relative_Offset",
        "Snp_Seq_No"
    ],
    values=cols,
    aggfunc={
        "Predicted_Revenue": np.sum,
        "Revenue": np.sum
    },
    margins=None,
).fillna(nan_value)
# final_merge_pivot1['End_of_Month'] = pd.to_datetime(final_merge_pivot1['End_of_Month']).dt.date
# final_merge_pivot1.reset_index(level=final_merge_pivot1.index.names)

# show_stats(revtime_pivot1)
final_merge_pivot1.sort_values(
    by=[ "Forecast_Date", "Snapshot_Date", "End_of_Month", "Relative_Offset","Snp_Seq_No"], inplace=True
)
# reorder columns
# cols_order = [0,4,1,2,3]
# final_merge_pivot1 = final_merge_pivot1[[final_merge_pivot1.columns[i] for i in cols_order]]

# set ALL float columns to '${:,.2f}' formatting (including the percentage)
# format_dict = {col_name: '${:,.1f}' for col_name in final_merge_pivot1.select_dtypes(float).columns}
# override the percentage column
format_dict = {col_name: "{:,}" for col_name in final_merge_pivot1.columns}
format_dict["Predicted_Revenue_Variance_Percent"] = "{:.2f}"


# Format with commas and round off to two decimal places in pandas
# final_merge_pivot1 = final_merge_pivot1[[['Snapshot_Date_Short','End_of_Month'] cols]]



In [ ]:
# Predicted Revenue by EOM for latest snapshot date and Seq
final_merge_pivot1.query(
    "Relative_Snapshot_Month_Offset == 0 and Relative_Offset >= 0"
)  # .tail(30)#.style.format(format_dict)#.style.format('{:,}'# .query('(End_of_Month == @current_eom)')



In [ ]:
# Create new forecasting file with new forecast date
'''
final_cols = [
    "Forecast_Date",
    "End_of_Month",
    "Relative_Month_Offset",
    "Snapshot_Date",
    "Relative_Snapshot_Month_Offset",
    "Relative_Offset",
    "Snp_Seq_No",
    "Sub_Region_Code",
    "Predicted_Revenue"
]
revenue_predict_new = final_merge_df.query(final_cond)[final_cols].copy()
revenue_predict_new = final_merge_df1[final_cols].copy()
'''
revenue_predict_new = final_merge_df1.query('Predicted_Revenue != 0')
revenue_predict_new = convert_date_cols(revenue_predict_new)
revenue_predict_new = revenue_predict_new.sort_values(by=sort_cols_snp).reset_index(
    drop=True
)
print("new:")
print(revenue_predict_new.info(verbose=True))
# reorder columns
# cols_order = [7,0,1,2,3,4,5,6]
# revenue_predict_new = revenue_predict_new[[revenue_predict_new.columns[i] for i in cols_order]]
filename = (
    output_data_path
    + "./revenue_forecast_"
    + str(pd.to_datetime(run_datetime).normalize().date())
    + ".csv"
)
revenue_predict = revenue_predict_new.copy()
revenue_predict = revenue_predict.sort_values(by=sort_cols_snp).reset_index(drop=True)
revenue_predict.to_csv(filename, header=True, index=False)
revenue_predict.tail()



In [ ]:
"""
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

df = revenue_predict.copy()
revenue_predict.query("Relative_Month_Offset >= 0").groupby("Relative_Offset").apply(
    lambda df: pd.Series(
        {
            "Mean Absolute Percent Error(MAPE)": MAPE(
                df[target_column_name], df["Predicted_Revenue"]
            ),
            "Root Mean Square Error (RMSE)": np.sqrt(
                mean_squared_error(df[target_column_name], df["Predicted_Revenue"])
            ),
            "Mean Absolute Error (MAE)": mean_absolute_error(
                df[target_column_name], df["Predicted_Revenue"]
            ),
        }
    )
)

"""

In [ ]:

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pd.set_option("display.float_format", lambda x: "%.2f" % x)

revenue_predict.query("Relative_Month_Offset < 0").groupby("Snapshot_Date").apply(
    lambda df: pd.Series(
        {
            "Mean Absolute Percent Error(MAPE)": MAPE(
                df[target_column_name], df["Predicted_Revenue"]
            ),
            "Root Mean Square Error (RMSE)": np.sqrt(
                mean_squared_error(df[target_column_name], df["Predicted_Revenue"])
            ),
            "Mean Absolute Error (MAE)": mean_absolute_error(
                df[target_column_name], df["Predicted_Revenue"]
            ),
        }
    )
)



In [ ]:
title = "Actual vs Predicted Revenue"

import altair as alt
#alt.themes.enable('fivethirtyeight') #default, latimes, ggplot2, fivethirtyeight, urbaninstitute
theme = alt.themes.enable('latimes')
source = final_merge_df.copy()
fontsize = 10
height = 450
width = 1000

x = 'yearmonth(End_of_Month):O'  # df['Importance'].astype(str)
y = 'sum(Predicted_Revenue):Q'  # df['Feature'].astype(str)
y1 = 'sum(Revenue):Q'
col = 'yearmonthdate(Snapshot_Date_Short):T' #.dt.date.astype(str) #'Snapshot_Date_Short'

ytransform ='datum.Predicted_Revenue / 1000000'
y1transform ='datum.Revenue / 1000000'


filter1 = "datum.Relative_Month_Offset >= 0 && datum.Relative_Snapshot_Month_Offset ==0"
filter2 = "datum.Relative_Month_Offset < 0"
filter3 = "datum.Relative_Offset == 0 && datum.Relative_Month_Offset < 0"

base = alt.Chart(source).encode(
        alt.X(x, axis=alt.Axis(title="End_of_Month"))
)

bars = base.mark_bar().encode(  #color='#5276A7', #fontSize=fontsize
    alt.Y('sum(Predicted_Revenue_million):Q', axis=alt.Axis(title="Predicted_Revenue(Millions)")),
    #color=alt.Color('legend:O', scale=alt.Scale(range=['grey', 'blue'], domain=['Revenue_Million1', 'Predicted_Revenue_Million1'])),
    tooltip=[alt.Tooltip(y, format=',.0f'),
             alt.Tooltip('End_of_Month', format='%b %Y')]
).transform_calculate(
  Predicted_Revenue_million=ytransform
).transform_filter(
  filter1
)
bars_text = bars.mark_text(
    align='center',
    baseline='bottom',
    #fontSize=fontsize,
    #dx=18,
    dy=15,  
    color='white'
).encode(
    text=alt.Text('sum(Predicted_Revenue_million):Q', format=',.1f')
).transform_calculate(
  Predicted_Revenue_million=ytransform
).transform_filter(
  filter1
)
#band = base.mark_errorband(extent="ci").encode(y=alt.Y("Predicted_Revenue:Q"))
bar = base.mark_bar(color='grey').encode(
    alt.Y('sum(Revenue_million):Q', axis=alt.Axis(title="Revenue(Millions)")),
    tooltip=[alt.Tooltip(y1, format=',.0f'),
             alt.Tooltip('End_of_Month', format='%b %Y')]
).transform_calculate(
  Revenue_million=y1transform
).transform_filter(
  filter3
)

#'''
bar_text = bar.mark_text(
    align='center',
    baseline='bottom',
    #fontSize=fontsize,
    #dx=18,
    dy=15,  
    color='white'
).encode(
    text=alt.Text('sum(Revenue_million):Q', format=',.1f')
).transform_calculate(
  Revenue_million=y1transform
).transform_filter(
  filter3
)
#'''
alt_layers = alt.layer(bar + bar_text, bars + bars_text).properties(height=height, width=width, title=title) #resolve_scale(y='independent').
alt_layers

In [ ]:
title = "Actual vs Predicted Revenue with Confidence Interval"
'''
import altair as alt
#alt.themes.enable('fivethirtyeight') #default, latimes, ggplot2, fivethirtyeight, urbaninstitute
theme = alt.themes.enable('latimes')

source = final_merge_df1.copy()
fontsize = 10
height = 450
width = 1000

x = 'yearmonth(End_of_Month):O'  # df['Importance'].astype(str)
y = 'sum(Predicted_Revenue):Q'  # df['Feature'].astype(str)
y1 = 'sum(Revenue):Q'
col = 'yearmonthdate(Snapshot_Date_Short):T' #.dt.date.astype(str) #'Snapshot_Date_Short'

ytransform ='datum.Predicted_Revenue / 1000000'
y1transform ='datum.Revenue / 1000000'


filter1 = "datum.Relative_Offset >= 0 && datum.Relative_Snapshot_Month_Offset == 0"
filter2 = "datum.Relative_Month_Offset < 0"
filter3 = "datum.Relative_Offset == 0 && datum.Relative_Month_Offset < 0"

base = alt.Chart(source).encode(
        alt.X(x, axis=alt.Axis(title="End_of_Month"))
)

lines = base.mark_line().encode(  #color='#5276A7', #fontSize=fontsize
    alt.Y('sum(Predicted_Revenue_million):Q', axis=alt.Axis(title="Predicted_Revenue(Millions)")),
    #color=alt.Color('legend:O', scale=alt.Scale(range=['grey', 'blue'], domain=['Revenue_Million1', 'Predicted_Revenue_Million1'])),
    tooltip=[alt.Tooltip(y, format=',.0f'),
             alt.Tooltip('End_of_Month', format='%b %Y')]
).transform_calculate(
  Predicted_Revenue_million=ytransform
).transform_filter(
  filter1
)
lines_text = lines.mark_text(
    align='center',
    baseline='bottom',
    #fontSize=fontsize,
    #dx=18,
    dy=-15,  
    #color='white'
).encode(
    text=alt.Text('sum(Predicted_Revenue_million):Q', format=',.1f')
).transform_calculate(
  Predicted_Revenue_million=ytransform
).transform_filter(
  filter1
)
band = base.mark_errorband(extent="ci"
).encode(y=alt.Y('Predicted_Revenue_million:Q')
).transform_calculate(
  Predicted_Revenue_million=ytransform
)

line = base.mark_line(color='grey').encode(
    alt.Y('sum(Revenue_million):Q', axis=alt.Axis(title="Revenue(Millions)")),
    tooltip=[alt.Tooltip(y1, format=',.0f'),
             alt.Tooltip('End_of_Month', format='%b %Y')]
).transform_calculate(
  Revenue_million=y1transform
).transform_filter(
  filter3
)
line_text = line.mark_text(
    align='center',
    baseline='bottom',
    #fontSize=fontsize,
    #dx=18,
    dy=-15,  
    #color='white'
).encode(
    text=alt.Text('sum(Revenue_million):Q', format=',.1f')
).transform_calculate(
  Revenue_million=y1transform
).transform_filter(
  filter3
)

alt_layers = alt.layer(line + line_text, lines + lines_text, band).properties(height=height, width=width, title=title) #resolve_scale(y='independent').
alt_layers
'''

In [ ]:
title = "Actual vs Predicted Revenue with Facets by Snapshot_Date"
'''
source = revenue_predict.copy()
fontsize = 10
height = 375
width = 500

x = "yearmonth(End_of_Month):O"  # df['Importance'].astype(str)
y = "Predicted_Revenue:Q"  # df['Feature'].astype(str)
y1 = "Revenue:Q"
col = "yearmonth(Snapshot_Date):T"  # .dt.date.astype(str) #'Snapshot_Date_Short'

ytransform = "datum.Predicted_Revenue / 1000000"
y1transform = "datum.Revenue / 1000000"

base = alt.Chart(source).encode(x=x)

bars = (
    base.mark_bar()
    .encode(  # color='#5276A7',
        alt.Y("Predicted_Revenue_million:Q", axis=alt.Axis(titleColor="grey")),
        tooltip=[
            alt.Tooltip(y, format=",.0f"),
            alt.Tooltip("End_of_Month", format="%b %Y"),
        ],
    )
    .transform_calculate(Predicted_Revenue_million=ytransform)
    .properties(width=width, height=height)
)



line = (
    base.mark_line(color="#F18727")
    .encode(
        alt.Y("Revenue_million:Q", axis=alt.Axis(titleColor="#F18727")),
        tooltip=[
            alt.Tooltip(y1, format=",.0f"),
            alt.Tooltip("End_of_Month", format="%b %Y"),
        ],
    )
    .transform_calculate(Revenue_million=y1transform)
)

text = (
    bars.mark_text(
        align="center",
        baseline="bottom",
        # fontSize=fontsize,
        # dx=2,
        dy=15,
        color="white",
    )
    .encode(text=alt.Text("Predicted_Revenue_million:Q", format=",.1f"))
    .transform_calculate(Predicted_Revenue_million=ytransform)
)

alt.layer(bars + text, line, data=source).facet(facet=col, columns=2, title=title)
'''

In [ ]:
title = "Predicted Revenue Variance Percent with Error Bars"
'''
source = (
    revenue_predict.copy()
)  # final_merge_df.query('Relative_Month_Offset >= -11').copy()
fontsize = 10
height = 600
width = 800

x = "yearmonth(End_of_Month):O"  # df['Importance'].astype(str)
y = "mean(Predicted_Revenue):Q"  # df['Feature'].astype(str)
y1 = "mean(Revenue):Q"
col = "yearmonth(Snapshot_Date):T"  # .dt.date.astype(str) #'Snapshot_Date_Short'

ytransform = "datum.Predicted_Revenue / 1000"
y1transform = "datum.Revenue / 1000"

base = alt.Chart(source).encode(x=x)

bars = (
    base.mark_bar()
    .encode(  # color='#5276A7',
        alt.Y(y),
        tooltip=[
            alt.Tooltip(y, format=",.2f"),
            alt.Tooltip("End_of_Month", format="%b %Y"),
        ],
    )
    .properties(width=width, height=height)
)

error_bars = base.mark_errorbar(extent="ci").encode(y=alt.Y("Predicted_Revenue:Q"))

line = (
    base.mark_line(color="#F18727")
    .encode(
        alt.Y('mean(Revenue_1000s):Q', axis=alt.Axis(titleColor="#F18727")),
        tooltip=[
            alt.Tooltip('mean(Revenue_1000s):Q', format=",.2f"),
            alt.Tooltip("End_of_Month", format="%b %Y"),
        ],
    )
    .transform_calculate(Revenue_1000s=y1transform)
)

text = (
    bars.mark_text(
        align="center",
        baseline="bottom",
        # fontSize=fontsize,
        # dx=2,
        dy=15,
        color='white'
    )
    .encode(text=alt.Text("mean(Predicted_Revenue_1000s):Q", format=",.2f"))
    .transform_calculate(Predicted_Revenue_1000s=ytransform)
)

alt.layer(bars + error_bars + text, line, data=source).properties(height=height, width=width, title=title)
'''

In [ ]:

from azureml.interpret import ExplanationClient
# Get model explanation data

#while best_run.get_status() not in ['Completed','Failed']: 
#    print('Run {} not in terminal state'.format(best_run.id))
#    time.sleep(10) 
    
feature_plot = 0
if ExplanationClient.from_run(best_run) is not None:
    try:
        client = ExplanationClient.from_run(best_run)
        engineered_explanations = client.download_model_explanation(raw=False)
        #print(engineered_explanations.get_feature_importance_dict())
        print(
            "You can visualize the engineered explanations under the 'Explanations (preview)' tab in the AutoML run at:-\n"
            + best_run.get_portal_url()
        )
        raw_explanations = client.download_model_explanation(raw=True)
        # print(raw_explanations.get_feature_importance_dict())
        print(
            "You can visualize the raw explanations under the 'Explanations (preview)' tab in the AutoML run at:-\n"
            + best_run.get_portal_url()
        )

        feature_imp_dict = pd.DataFrame(
            raw_explanations.get_feature_importance_dict().items()
        )
        feature_imp_dict.columns = ["Feature", "Importance"]
    
        feature_imp_dict.plot()
    
    except Exception as error:
        print(error)
        log_error("{} {}".format(notebook, error)) #log error in sentry
        #raise dbutils.notebook.exit(error) #raise the exception

feature_plot

In [ ]:

# Print feature importance

import altair as alt

alt.data_transformers.disable_max_rows()

# alt.themes.enable('fivethirtyeight') #default, latimes, ggplot2, fivethirtyeight, urbaninstitute

theme = alt.themes.enable("latimes")
theme.fontsize = 14

imp_chart = alt.Chart(revenue_predict).mark_bar()

if feature_imp_dict is not None:
    try:
        source = feature_imp_dict
        # display(source)
        title = "Forecast Model Drivers"

        fontsize = 12
        height = 600
        width = 600

        bars = (
            alt.Chart(source)
            .mark_bar()
            .encode(x="Importance:Q", y=alt.Y("Feature:O", sort="-x"))
        )  # fontSize=fontsize

        text = bars.mark_text(align="left", baseline="middle", dx=3).encode(
            text=alt.Text("Importance:Q", format=",.0f")
        )  #    fontSize=fontsiz

        imp_chart = (bars + text).properties(title=title)
    
    except Exception as error:
        print(error)
        log_error("{} {}".format(notebook, error)) #log error in sentry
        raise dbutils.notebook.exit(error) #raise the exception

imp_chart

In [ ]:
#**********************************************************DONE*************************************************************

In [ ]:
'''
#Operationalization means getting the model into the cloud so that other can run it after you close the notebook. We will create a docker running on Azure Container Instances with the model.
description = 'AutoML Revenue forecaster'
tags = None
model = remote_run.register_model(model_name = model_name, description = description, tags = tags)

print(remote_run.model_id)

#Develop the scoring script
#For the deployment we need a function which will run the forecast on serialized data. It can be obtained from the best_run.
script_file_name = 'score_fcast.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file_name)

#Deploy the model as a Web Service on Azure Container 
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model

inference_config = InferenceConfig(environment = best_run.get_environment(), 
                                   entry_script = script_file_name)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 2, 
                                               tags = {'type': 'automl-forecasting'},
                                               description = 'Automl forecasting revenue service')

aci_service_name = 'automl-revenue-forecast-01'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

aci_service.get_logs()

#Call the service
import json
X_query = X_test.copy()
# We have to convert datetime to string, because Timestamps cannot be serialized to JSON.
X_query[time_column_name] = X_query[time_column_name].astype(str)
# The Service object accept the complex dictionary, which is internally converted to JSON string.
# The section 'data' contains the data frame in the form of dictionary.
test_sample = json.dumps({'data': X_query.to_dict(orient='records')})
response = aci_service.run(input_data = test_sample)
# translate from networkese to datascientese
try: 
    res_dict = json.loads(response)
    y_fcst_all = pd.DataFrame(res_dict['index'])
    y_fcst_all[time_column_name] = pd.to_datetime(y_fcst_all[time_column_name], unit = 'ms')
    y_fcst_all['forecast'] = res_dict['forecast']    
except:
    print(res_dict)
    
y_fcst_all.head()    

#Delete the web service if desired
#serv = Webservice(ws, 'automl-revenue-forecast-01')
#serv.delete()     # don't do it accidentally
'''